# Использование графов знаний для RAG

### Принеобходимости установи все пакеты
Лучше создать новое окружение, так же рекомендую создать новую папку и закинуть туда ноутбук (чтобы сгенерированные файлы не потерять)

In [1]:
%pip install llama-index-llms-langchain
!pip install gigachain -q
%pip install llama-index-readers-file
!pip install llama-index
!pip install faiss-cpu
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor
!pip install pyvis

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     -------------------------- ------------- 41.0/61.0 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/125.2 kB ? eta -:--:--
     ---------------------------------- --- 112.6/125.2 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 125.2/125.2 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
     ---------------------------------------- 51.6/51.6 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------

In [2]:
%pip install llama-index-llms-ollama

Note: you may need to restart the kernel to use updated packages.


### Импортирование необходимых методов

In [2]:
from getpass import getpass
from langchain.chat_models.gigachat import GigaChat
from llama_index.llms.langchain import LangChainLLM
from llama_index.llms.ollama import Ollama

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.node_parser import SemanticSplitterNodeParser


In [3]:
gigachat_api_key = getpass(prompt='Введите API ключ от GigaChat')

Введите API ключ от GigaChat ········


### Загрузка файлов из папки
Файлы загружаются постранично, т.е. каждый элемент documents - отдельная страница документа. (по порядку файлов в папке и страниц в документе) 

In [48]:
reader = SimpleDirectoryReader(input_dir="C:/Users/MainUser/Desktop/JupyterNotebooks/kg_llm_rag/NPA")
documents = reader.load_data()
#len(documents), type(documents), documents

### Проверка загрузки файлов
Название файла, кол-во страниц

In [49]:
def check_loaders(documents):
    info_docs = []
    current_doc_name = ''
    max_page_label = 0
    for doc in documents:
        if doc.metadata['file_name'] != current_doc_name:
            if current_doc_name != '':
                info_docs.append([current_doc_name, max_page_label])
            current_doc_name, max_page_label = doc.metadata['file_name'], doc.metadata['page_label']
        else:
            max_page_label = doc.metadata['page_label']
    info_docs.append([current_doc_name, max_page_label])
    
    return info_docs

info_docs = check_loaders(documents)
info_docs, len(info_docs)

([['FSB196.pdf', '19'],
  ['FSB281.pdf', '5'],
  ['FSB282.pdf', '7'],
  ['FSB366.pdf', '6'],
  ['FSB367.pdf', '4'],
  ['FSB368.pdf', '9'],
  ['FSTEK235.pdf', '12'],
  ['FSTEK236.pdf', '13'],
  ['FSTEK239.pdf', '34'],
  ['FSTEK75.pdf', '10'],
  ['FZ187.pdf', '17'],
  ['PP127.pdf', '21'],
  ['PP1272.pdf', '15'],
  ['PP1478.pdf', '18'],
  ['PP162.pdf', '13'],
  ['PP1912.pdf', '51'],
  ['PP2461.pdf', '24'],
  ['PP743.pdf', '4'],
  ['YK166.pdf', '3'],
  ['YK250.pdf', '3']],
 20)

### Сократим кол-во документов
Здесь берется 5 страниц первого документа.
Делаем мы это с целью искусственно уменьшить кол-во потраченных токенов.
Для тестового прогона не стоит устанавливать больше 19)

In [50]:
documents = [documents[i] for i in range(int(input("Введите кол-во страниц документа: ")))]

Введите кол-во страниц документа:  288


In [51]:
import re

In [52]:
from llama_index.core.node_parser import SemanticSplitterNodeParser, SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from sentence_transformers import SentenceTransformer
from llama_index.core import Settings

from llama_index.core import PropertyGraphIndex

### Предобработаем тексты документов

In [53]:
def text_processing(documents):
    all_text_corpus = []
    for doc in documents:
        current_text = re.sub(r'[^А-Яа-яЁё0-9]',' ',doc.text)
        current_text = re.sub(r'\s+',' ',current_text)
        all_text_corpus.append(current_text.lower())
    return all_text_corpus

all_text = text_processing(documents)

for i in range(len(documents)):
    documents[i].text = all_text[i]

documents[1].text, len(documents), len(all_text)

('приложение требования к средствам предназначенным для обнаружения предупреждения и ликвидации последствий компьютерных атак и реагирования на компьютерные инциденты приложение к приказу фсб россии от 6 мая 2019 года 196 общие положения 1 настоящие требования определяют требования к устанавливаемым и используемым на всей территории российской федерации в дипломатических представительствах и или консульских учреждениях российской федерации техническим программным программно аппаратным и иным средствам для обнаружения в том числе для поиска признаков компьютерных атак в сетях электросвязи используемых для организации взаимодействия объектов критической информационной инфраструктуры российской федерации далее критическая информационная инфраструктура предупреждения ликвидации последствий компьютерных атак и или обмена информацией необходимой субъектам критической информационной инфраструктуры при обнаружении предупреждении и или ликвидации последствий компьютерных атак а также криптограф

### Укажем сплитер

In [67]:
embed = HuggingFaceEmbedding()
dir(embed)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '__weakref__',
 '_abc_impl',
 '_aget_query_embedding',
 '_aget_text_embed

In [75]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/LaBSE"
)

node_parser = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

base_splitter = SentenceSplitter(chunk_size=512)

In [76]:
import nest_asyncio

nest_asyncio.apply()

### Загрузим GigaChat

In [77]:
giga_llm = LangChainLLM(llm=GigaChat(credentials=gigachat_api_key, verify_ssl_certs=False, temperature=0.01))

In [78]:
Settings.llm = Ollama(model="llama3", request_timeout=180.0, temperature=0.2)

## Все шаги снизу, до указателя, проводить только при создании графа знаний.
Лучше подгрузить из файлов, с помощью кода после указателя.

### Укажем способ сохранения графа
Наш граф находится в папке ./storage

In [79]:
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext, ServiceContext

#Settings.context_window=4096

#Settings.num_output = 256

service_context = ServiceContext.from_defaults(
    llm=Settings.llm,
    #node_parser=node_parser,
    node_parser=base_splitter,
    chunk_size = 512,
    embed_model=embed_model,
)

storage_dir = input('Введите путь для сохранения графа, стандартный путь: ./storage, в случае тестового прогона рекомендую ./storage_test')

graph_store = SimpleGraphStore(persist=storage_dir)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

type(service_context), type(graph_store), type(storage_context)

C:\Users\MainUser\AppData\Local\Temp\ipykernel_2208\1171506425.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


Введите путь для сохранения графа, стандартный путь: ./storage, в случае тестового прогона рекомендую ./storage_test ./storage_test


(llama_index.core.service_context.ServiceContext,
 llama_index.core.graph_stores.simple.SimpleGraphStore,
 llama_index.core.storage.storage_context.StorageContext)

### Выделим из текстов узлы и ребра графа знаний


Функция для первичного выделения триплетов из текста

In [59]:
#def fn_extract_simple_triplets():
    

In [ ]:
from llama_index.core import KnowledgeGraphIndex
from llama_index.core.prompts import PromptTemplate, PromptType


kg_llm_exactor_tmpl = (
    "Ниже приведен небольшой текст. Возьмите этот текст и извлеките из него до "
    "{max_knowledge_triplets} "
    "триплетов графа знаний в формате (субъект, связь, объект).\n"
    "---------------------\n"
    "Например: "
    "Текст: Обеспечение безопасности значимых объектов осуществляется в соответствии с законодательством Российской Федерации о государственной тайне."
    "Триплеты:\n"
    "(Обеспечение безопасности значимых объектов, осуществляется в соответствии с, законодательством Российской Федерации о государственной тайне)\n"
    "Текст: Безопасность значимых объектов обеспечивается субъектами критической информационной инфраструктуры в рамках функционирования систем безопасности значимых объектов, создаваемых субъектами критической информационной инфраструктуры в соответствии со статьей 10 Федерального закона от 26 июля 2017 г.187-ФЗ \"О безопасности критической информационной инфраструктуры Российской Федерации\"\n"
    "Триплеты:\n"
    "(Безопасность значимых объектов, обеспечивается, субъектами критической информационной инфраструктуры)\n"
    "(Безопасность значимых объектов, обеспечивается, в рамках функционирования систем безопасности значимых объектов)\n"
    "(Значимые обЪекты, имеют, системы безопасности)\n"
    "(Системы безопасности, создаются, субъектами критической информационной инфраструктуры)\n"
    "(Системы безопасности, создаются в соответствии со, статьей 10 Федерального закона от 26 июля 2017 г.187-ФЗ \"О безопасности критической информационной инфраструктуры Российской Федерации\")\n"
    "---------------------\n"
    "Текст: {text}\n"
    "Триплеты:\n"
)

kg_llm_exactor_prompt = PromptTemplate(
    kg_llm_exactor_tmpl,
    prompt_type=PromptType.KNOWLEDGE_TRIPLET_EXTRACT,
)


kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=20,
    kg_triplet_extract_template=kg_llm_exactor_prompt,
    embed_model=embed_model,
    llm=Settings.llm,
    include_embeddings=True,
    show_progress=True,
    storage_context=storage_context,
    service_context=service_context
)

storage_context.persist(storage_dir)

kg_index, type(kg_index)

Parsing nodes:   0%|          | 0/288 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/680 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/16 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/15 [00:00<?, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

...

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/11 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/12 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/13 [00:00<?, ?it/s]

### Визуализация графа знаний
Сам граф находится в папке /storage

In [28]:
from pyvis.network import Network
import networkx as nx
from IPython.display import display, HTML


g = kg_index.get_networkx_graph(limit=100000)
network = Network(notebook=True, cdn_resources="in_line", directed=True)
network.from_nx(g)
#network.show("kg.html")

html = network.generate_html()
with open("kg_test.html", mode='w', encoding='utf-8') as fp:
        fp.write(html)
#display(HTML(html))

print(g)

Graph with 307 nodes and 215 edges


## Далее работа с созданным графом

### Загрузим граф знаний
Если только что его создали - не выполнять.

In [16]:
service_context = ServiceContext.from_defaults(
    llm=Settings.llm,
    #node_parser=node_parser,
    node_parser=base_splitter,
    chunk_size = 512,
    embed_model=embed_model
)

graph_store = SimpleGraphStore.from_persist_dir(persist_dir="./storage")

storage_context = StorageContext.from_defaults(graph_store=graph_store)

C:\Users\MainUser\AppData\Local\Temp\ipykernel_2208\3304024439.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


### Объявим ретривер

Запасной, не рабочий)

from llama_index.core.indices.property_graph import LLMSynonymRetriever

prompt = (
    "Представь что ты специалист по информационной безопасности. Посмотри на ключевые слова и ответь на вопрос"
    "----\n"
    "Вопрос: {query_str}\n"
    "----\n"
    "Ключевые слова: "
)



synonym_retriever = LLMSynonymRetriever(
    index.property_graph_store,
    llm=llm,
    include_text=True,
    synonym_prompt=prompt,
    max_keywords=10,
    path_depth=1,
)

retriever = index.as_retriever(sub_retrievers=[synonym_retriever])

Основной

In [21]:
!pip install llama-index-llms-fireworks

In [22]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=giga_llm,
    include_text=True,
    #entity_extract_policy='intersection',
    verbose=True
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever
)

#К тестовому варианту запросов (нужно другое хранилище, или класс переписать)
#graph_query_engine = KnowledgeGraphQueryEngine(
#    storage_context=storage_context,
#    service_context=service_context,
#    verbose=True
#)

C:\Users\MainUser\AppData\Local\Temp\ipykernel_2208\3487671977.py:4: DeprecationWarning: Call to deprecated class KnowledgeGraphRAGRetriever. (KnowledgeGraphRAGRetriever is deprecated, it is recommended to use PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  graph_rag_retriever = KnowledgeGraphRAGRetriever(


### Запросы к GigaChat

In [23]:
from IPython.display import display, Markdown

response = query_engine.query(
    "Что такое госсопка?"
)

display(Markdown(f"<b>{response}</b>"))

D:\Dmitriy\Anaconda\envs\kg_llm_rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


<b>Empty Response</b>

Тестовый(пока не рабочий)

In [71]:
graph_query = query_engine.generate_query(
    "Что такое госсопка?",
)

graph_query = graph_query.replace("WHERE", "\n  WHERE").replace(
    "RETURN", "\nRETURN"
)

display(
    Markdown(
        f"""
```cypher
{graph_query}
```
"""
    )
)

AttributeError: 'RetrieverQueryEngine' object has no attribute 'generate_query'